# Model Training with Amazon SageMaker Autopilot
Yuanyuan Sun, 11/27/2023

Utilize Amazon SageMaker Autopilot for training a BERT-based natural language processing (NLP) model. The model is designed to analyze customer feedback, classifying messages into positive (1), neutral (0), and negative (-1) sentiment.

In [3]:
import boto3
import sagemaker
import pandas as pd
import numpy as np
import botocore
import time
import json

config = botocore.config.Config(user_agent_extra='')

sm = boto3.client(service_name='sagemaker', config=config)
sm_runtime = boto3.client('sagemaker-runtime', config=config)
sess = sagemaker.Session(sagemaker_client=sm, sagemaker_runtime_client=sm_runtime)

bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = sess.boto_region_name

/opt/conda/lib/python3.7/site-packages/boto3/compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/

  warnings.warn(warning, PythonDeprecationWarning)


# 1. Load Transformed Data Set

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [6]:
path = './womens_clothing_ecommerce_reviews_balanced.csv'

df = pd.read_csv(path, delimiter=',')
df.head()

,sentiment,review_body,product_category
0,-1,This suit did nothing for me. the top has zero...,Swim
1,-1,Like other reviewers i saw this dress on the ...,Dresses
2,-1,I wish i had read the reviews before purchasin...,Knits
3,-1,I ordered these pants in my usual size (xl) an...,Legwear
4,-1,I noticed this top on one of the sales associa...,Knits


In [7]:
path_autopilot = './womens_clothing_ecommerce_reviews_balanced_for_autopilot.csv'

df[['sentiment', 'review_body']].to_csv(path_autopilot, sep=',', index=False)

# 2. Set up the Autopilot job

In [8]:
# Upload data to S3 bucket
autopilot_train_s3_uri = sess.upload_data(bucket=bucket, key_prefix='autopilot/data', path=path_autopilot)
autopilot_train_s3_uri

's3://sagemaker-us-east-1-594527930436/autopilot/data/womens_clothing_ecommerce_reviews_balanced_for_autopilot.csv'

In [9]:
# Verify the presence of the dataset in this S3 bucket folder:
!aws s3 ls $autopilot_train_s3_uri

2023-11-28 01:49:11    2253749 womens_clothing_ecommerce_reviews_balanced_for_autopilot.csv


In [10]:
# Specify the S3 output path for generated assets
# This path encompasses Autopilot outputs, including Jupyter notebooks (analysis), Python scripts (feature engineering), and trained models.
model_output_s3_uri = 's3://{}/autopilot'.format(bucket)
print(model_output_s3_uri)

s3://sagemaker-us-east-1-594527930436/autopilot


In [11]:
# Generate the name for the Autopilot job.
import time

timestamp = int(time.time())
auto_ml_job_name = 'automl-dm-{}'.format(timestamp)

In [12]:
# Configure the Autopilot job.
max_candidates = 3

automl = sagemaker.automl.automl.AutoML(    
    target_attribute_name='sentiment', 
    base_job_name=auto_ml_job_name, 
    output_path=model_output_s3_uri, 
    
    max_candidates=max_candidates,
    sagemaker_session=sess,
    role=role,
    max_runtime_per_training_job_in_seconds=1200,
    total_job_runtime_in_seconds=7200
)

# 3. Initiate the Autopilot job

In [13]:
automl.fit(    
    autopilot_train_s3_uri,     
    job_name=auto_ml_job_name, 
    wait=False, 
    logs=False
)

# 4. Monitor the Progress of the Autopilot Job

After initiating the Autopilot job, monitor its progress directly from the notebook using the SDK capabilities.

In [14]:
job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)

In [15]:
# Track the job progress

while 'AutoMLJobStatus' not in job_description_response.keys() and 'AutoMLJobSecondaryStatus' not in job_description_response.keys():
    job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
    print('[INFO] Autopilot job has not yet started. Please wait. ')
    # function `json.dumps` encodes JSON string for printing.
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for Autopilot job to start...')
    sleep(15)

print('[OK] AutoML job started.')

[OK] AutoML job started.


In [16]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/">processing jobs</a></b>'.format(region)))


Here we will use the same scheme as above to check the completion of the data analysis step. This step can be identified with the (primary) job status value `InProgress` and secondary job status values `Starting` and then `AnalyzingData`.

In [17]:
%%time

job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']

if job_status not in ('Stopped', 'Failed'):
    while job_status in ('InProgress') and job_sec_status in ('Starting', 'AnalyzingData'):
        job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        time.sleep(15)
    print('[OK] Data analysis phase completed.\n')
    
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress AnalyzingData

InProgress A

After completing data analysis, SageMaker AutoPilot produces two notebooks: 
- Data Exploration
- Candidate Definition

These notebooks are part of the AutoML job artifacts generated during the run. Before verifying the existence of the notebooks, ensure that the artifacts have been generated.


In [19]:
# Verify whether the Autopilot job artifacts have been generated.
# get the information about the running Autopilot job
job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name) 

# keep in the while loop until the Autopilot job artifacts will be generated
while 'AutoMLJobArtifacts' not in job_description_response.keys(): 
    # update the information about the running Autopilot job
    job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name) 
    
    print('[INFO] Autopilot job has not yet generated the artifacts. Please wait. ')
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for AutoMLJobArtifacts...')
    time.sleep(15)

print('[OK] AutoMLJobArtifacts generated.')

[OK] AutoMLJobArtifacts generated.


In [20]:
# Verify if the notebooks have been created.
# get the information about the running Autopilot job
job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name) 

# keep in the while loop until the notebooks will be created
while 'DataExplorationNotebookLocation' not in job_description_response['AutoMLJobArtifacts'].keys(): 
    # update the information about the running Autopilot job
    job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name) 
    
    print('[INFO] Autopilot job has not yet generated the notebooks. Please wait. ')
    print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))
    print('[INFO] Waiting for DataExplorationNotebookLocation...')
    time.sleep(15)

print('[OK] DataExplorationNotebookLocation found.')

[OK] DataExplorationNotebookLocation found.


Examine the generated resources directly in S3. By following the link, we can locate the notebooks in the 'notebooks' folder and download them by selecting the object and choosing 'Actions' -> 'Download'.


In [21]:
from IPython.core.display import display, HTML

generated_resources = job_description_response['AutoMLJobArtifacts']['DataExplorationNotebookLocation']
download_path = generated_resources.rsplit('/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb')[0]
job_id = download_path.rsplit('/', 1)[-1]

if not job_id: 
    print('No AutoMLJobArtifacts found.')
else: 
    display(HTML('<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/autopilot/{}/sagemaker-automl-candidates/{}/">generated notebooks</a> in S3 bucket</b>'.format(bucket, auto_ml_job_name, job_id)))

<a name='c1w3-5.'></a>
# 5. Feature engineering

Verify the completion of the feature engineering step.

In [22]:
%%time

job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    
    while job_status in ('InProgress') and job_sec_status in ('FeatureEngineering'):
    
        job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        time.sleep(5)
    print('[OK] Feature engineering phase completed.\n')
    
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

InProgress

FeatureEngineering

[OK] Feature engineering phase completed.



{

    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:594527930436:automl-job/automl-dm-1701136158",

    "AutoMLJobArtifacts": {

        "CandidateDefinitionNotebookLocation": "s3://sagemaker-us-east-1-594527930436/autopilot/automl-dm-1701136158/sagemaker-automl-candidates/automl-dm-1701136158-pr-1-e556da6192b247d0bddbb2441d7ea053fe96e/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",

        "DataExplorationNotebookLocation": "s3://sagemaker-us-east-1-594527930436/autopilot/automl-dm-1701136158/sagemaker-automl-candidates/automl-dm-1701136158-pr-1-e556da6192b247d0bddbb2441d7ea053fe96e/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"

    },

    "AutoMLJobConfig": {

        "CompletionCriteria": {

            "MaxAutoMLJobRuntimeInSeconds": 7200,

            "MaxCandidates": 3,

            "MaxRuntimePerTrainingJobInSeconds": 1200

        },

        "SecurityConfig": {

        

# 6. Training and Tuning the Model
Upon launching the Autopilot job, we specified the generation and comparison of three model candidates. 
As a result, we should observe three (3) SageMaker training jobs listed below.


In [33]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/hyper-tuning-jobs/">hyper-parameter tuning jobs</a></b>'.format(region)))


Verify the completion of the model tuning step.

In [24]:
%%time

job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
print(job_status)
print(job_sec_status)
if job_status not in ('Stopped', 'Failed'):
    
    while job_status in ('InProgress') and job_sec_status in ('ModelTuning'): 
        job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print(job_status, job_sec_status)
        time.sleep(5)
    print('[OK] Model tuning phase completed.\n')
    
print(json.dumps(job_description_response, indent=4, sort_keys=True, default=str))

InProgress

FeatureEngineering

[OK] Model tuning phase completed.



{

    "AutoMLJobArn": "arn:aws:sagemaker:us-east-1:594527930436:automl-job/automl-dm-1701136158",

    "AutoMLJobArtifacts": {

        "CandidateDefinitionNotebookLocation": "s3://sagemaker-us-east-1-594527930436/autopilot/automl-dm-1701136158/sagemaker-automl-candidates/automl-dm-1701136158-pr-1-e556da6192b247d0bddbb2441d7ea053fe96e/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb",

        "DataExplorationNotebookLocation": "s3://sagemaker-us-east-1-594527930436/autopilot/automl-dm-1701136158/sagemaker-automl-candidates/automl-dm-1701136158-pr-1-e556da6192b247d0bddbb2441d7ea053fe96e/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb"

    },

    "AutoMLJobConfig": {

        "CompletionCriteria": {

            "MaxAutoMLJobRuntimeInSeconds": 7200,

            "MaxCandidates": 3,

            "MaxRuntimePerTrainingJobInSeconds": 1200

        },

        "SecurityConfig": {

            "En

Finally, we can check the completion of the Autopilot job looking for the `Completed` job status.

In [25]:
%%time

from pprint import pprint

job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
pprint(job_description_response)
job_status = job_description_response['AutoMLJobStatus']
job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
print('Job status:  {}'.format(job_status))
print('Secondary job status:  {}'.format(job_sec_status))
if job_status not in ('Stopped', 'Failed'):
    while job_status not in ('Completed'):
        job_description_response = automl.describe_auto_ml_job(job_name=auto_ml_job_name)
        job_status = job_description_response['AutoMLJobStatus']
        job_sec_status = job_description_response['AutoMLJobSecondaryStatus']
        print('Job status:  {}'.format(job_status))
        print('Secondary job status:  {}'.format(job_sec_status))        
        time.sleep(10)
    print('[OK] Autopilot job completed.\n')
else:
    print('Job status: {}'.format(job_status))
    print('Secondary job status: {}'.format(job_status))

{'AutoMLJobArn': 'arn:aws:sagemaker:us-east-1:594527930436:automl-job/automl-dm-1701136158',

 'AutoMLJobArtifacts': {'CandidateDefinitionNotebookLocation': 's3://sagemaker-us-east-1-594527930436/autopilot/automl-dm-1701136158/sagemaker-automl-candidates/automl-dm-1701136158-pr-1-e556da6192b247d0bddbb2441d7ea053fe96e/notebooks/SageMakerAutopilotCandidateDefinitionNotebook.ipynb',

                        'DataExplorationNotebookLocation': 's3://sagemaker-us-east-1-594527930436/autopilot/automl-dm-1701136158/sagemaker-automl-candidates/automl-dm-1701136158-pr-1-e556da6192b247d0bddbb2441d7ea053fe96e/notebooks/SageMakerAutopilotDataExplorationNotebook.ipynb'},

 'AutoMLJobConfig': {'CompletionCriteria': {'MaxAutoMLJobRuntimeInSeconds': 7200,

                                            'MaxCandidates': 3,

                                            'MaxRuntimePerTrainingJobInSeconds': 1200},

                     'SecurityConfig': {'EnableInterContainerTrafficEncryption': False}},

 'Aut

Before moving to the next section make sure the status above indicates `Autopilot job completed`.

## 6.1. Compare model candidates

After completing model tuning, we can review all the candidates (pipeline evaluations with various hyperparameter combinations) explored by AutoML and organize them based on their final performance metric.

In [26]:
# Enumerate Autopilot-generated candidates, arranged in descending order based on accuracy.
candidates = automl.list_candidates(
    
    job_name=auto_ml_job_name, 
    sort_by='FinalObjectiveMetricValue' 
    
)

In [ ]:
# put the candidate existence check into the loop
while candidates == []:
    candidates = automl.list_candidates(job_name=auto_ml_job_name)
    print('[INFO] Autopilot job is generating the candidates. Please wait.')
    time.sleep(10)

print('[OK] Candidates generated.') 

print(candidates[0].keys())

dict_keys(['CandidateName', 'FinalAutoMLJobObjectiveMetric', 'ObjectiveStatus', 'CandidateSteps', 'CandidateStatus', 'InferenceContainers', 'CreationTime', 'EndTime', 'LastModifiedTime', 'CandidateProperties'])


The `CandidateName` holds the candidate name, and the `FinalAutoMLJobObjectiveMetric` element contains metric information, enabling the identification of the best candidate later. Let's verify that they have been generated.

In [29]:
while 'CandidateName' not in candidates[0]:
    candidates = automl.list_candidates(job_name=auto_ml_job_name)
    print('[INFO] Autopilot job is generating CandidateName. Please wait. ')
    sleep(10)

print('[OK] CandidateName generated.')

[OK] CandidateName generated.


In [30]:
while 'FinalAutoMLJobObjectiveMetric' not in candidates[0]:
    candidates = automl.list_candidates(job_name=auto_ml_job_name)
    print('[INFO] Autopilot job is generating FinalAutoMLJobObjectiveMetric. Please wait. ')
    sleep(10)

print('[OK] FinalAutoMLJobObjectiveMetric generated.')

[OK] FinalAutoMLJobObjectiveMetric generated.


In [31]:
print(json.dumps(candidates, indent=4, sort_keys=True, default=str))

[

    {

        "CandidateName": "automl-dm-17011361583CrgHRI7oc5M-001-3da8196d",

        "CandidateProperties": {

            "CandidateArtifactLocations": {

                "Explainability": "s3://sagemaker-us-east-1-594527930436/autopilot/automl-dm-1701136158/documentation/explainability/output",

                "ModelInsights": "s3://sagemaker-us-east-1-594527930436/autopilot/automl-dm-1701136158/documentation/model_monitor/output"

            },

            "CandidateMetrics": [

                {

                    "MetricName": "F1macro",

                    "Set": "Validation",

                    "StandardMetricName": "F1macro",

                    "Value": 0.5687199831008911

                },

                {

                    "MetricName": "PrecisionMacro",

                    "Set": "Validation",

                    "StandardMetricName": "PrecisionMacro",

                    "Value": 0.5674899816513062

                },

                {

         

Print the names of the candidates with their metric values:

In [32]:
print("metric " + str(candidates[0]['FinalAutoMLJobObjectiveMetric']['MetricName']))

for index, candidate in enumerate(candidates):
    print(str(index) + "  " 
        + candidate['CandidateName'] + "  " 
        + str(candidate['FinalAutoMLJobObjectiveMetric']['Value']))

metric validation:accuracy

0  automl-dm-17011361583CrgHRI7oc5M-001-3da8196d  0.5742599964141846

1  automl-dm-17011361583CrgHRI7oc5M-003-9d3cf397  0.5417699813842773

2  automl-dm-17011361583CrgHRI7oc5M-002-fad7b5b6  0.4245699942111969


## 6.2. Review Best Candidate

Having successfully concluded the Autopilot job on the dataset and visualized the trials, we can now retrieve information about the best candidate model and review its details.

In [34]:
# Retrieve information about the best candidate job generated.
candidates = automl.list_candidates(job_name=auto_ml_job_name)

if candidates != []:
    best_candidate = automl.best_candidate(        
        job_name=auto_ml_job_name         
    )
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))

{

    "CandidateName": "automl-dm-17011361583CrgHRI7oc5M-001-3da8196d",

    "CandidateProperties": {

        "CandidateArtifactLocations": {

            "Explainability": "s3://sagemaker-us-east-1-594527930436/autopilot/automl-dm-1701136158/documentation/explainability/output",

            "ModelInsights": "s3://sagemaker-us-east-1-594527930436/autopilot/automl-dm-1701136158/documentation/model_monitor/output"

        },

        "CandidateMetrics": [

            {

                "MetricName": "F1macro",

                "Set": "Validation",

                "StandardMetricName": "F1macro",

                "Value": 0.5687199831008911

            },

            {

                "MetricName": "PrecisionMacro",

                "Set": "Validation",

                "StandardMetricName": "PrecisionMacro",

                "Value": 0.5674899816513062

            },

            {

                "MetricName": "Accuracy",

                "Set": "Validation",

               

Check the existence of the candidate name for the best candidate.

In [35]:
while 'CandidateName' not in best_candidate:
    best_candidate = automl.best_candidate(job_name=auto_ml_job_name)
    print('[INFO] Autopilot Job is generating BestCandidate CandidateName. Please wait. ')
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print('[OK] BestCandidate CandidateName generated.')  

[OK] BestCandidate CandidateName generated.


Check the existence of the metric value for the best candidate.

In [36]:
while 'FinalAutoMLJobObjectiveMetric' not in best_candidate:
    best_candidate = automl.best_candidate(job_name=auto_ml_job_name)
    print('[INFO] Autopilot Job is generating BestCandidate FinalAutoMLJobObjectiveMetric. Please wait. ')
    print(json.dumps(best_candidate, indent=4, sort_keys=True, default=str))
    sleep(10)

print('[OK] BestCandidate FinalAutoMLJobObjectiveMetric generated.')  

[OK] BestCandidate FinalAutoMLJobObjectiveMetric generated.


Print the information about the best candidate:

In [37]:
best_candidate_identifier = best_candidate['CandidateName']
print("Candidate name: " + best_candidate_identifier)
print("Metric name: " + best_candidate['FinalAutoMLJobObjectiveMetric']['MetricName'])
print("Metric value: " + str(best_candidate['FinalAutoMLJobObjectiveMetric']['Value']))

Candidate name: automl-dm-17011361583CrgHRI7oc5M-001-3da8196d

Metric name: validation:accuracy

Metric value: 0.5742599964141846


Below is the artifacts generated by Autopilot, I've downloaded some of the output documents and will submit them on Blackboard:
```
data-processor-models/        # "models" learned to transform raw data into features 
documentation/                # explainability and other documentation about the model
preprocessed-data/            # data for train and validation
sagemaker-automl-candidates/  # candidate models which autopilot compares
transformed-data/             # candidate-specific data for train and validation
tuning/                       # candidate-specific tuning results
validations/                  # validation results
```

In [38]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review all <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}?region={}&prefix=autopilot/{}/">output in S3</a></b>'.format(
            bucket, region, auto_ml_job_name
        )
    )
)

# 7. Deploy and Test Best Candidate Model

In [39]:
inference_response_keys = ['predicted_label', 'probability']

In [40]:
autopilot_model = automl.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.large',
    candidate=best_candidate,
    inference_response_keys=inference_response_keys,
    predictor_cls=sagemaker.predictor.Predictor,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer()
)

print('\nEndpoint name:  {}'.format(autopilot_model.endpoint_name))

--------!

Endpoint name:  sagemaker-sklearn-automl-2023-11-28-02-35-19-022


Review the SageMaker endpoint in the AWS console.

In [41]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">SageMaker REST endpoint</a></b>'.format(region, autopilot_model.endpoint_name)))

In [42]:
# Test the model
# Generate predictions for a few actual reviews using the deployed endpoint.

review_list = ['This product is great!',
               'OK, but not great.',
               'This is not the right product.']

for review in review_list:
    
    # remove commas from the review since we're passing the inputs as a CSV
    review = review.replace(",", "")

    response = sm_runtime.invoke_endpoint(
        EndpointName=autopilot_model.endpoint_name, # endpoint name
        ContentType='text/csv', # type of input data
        Accept='text/csv', # type of the inference in the response
        Body=review # review text
        )

    response_body=response['Body'].read().decode('utf-8').strip().split(',')

    print('Review: ', review, ' Predicated class: {}'.format(response_body[0]))

print("(-1 = Negative, 0=Neutral, 1=Positive)")

Review:  This product is great!  Predicated class: 1

Review:  OK but not great.  Predicated class: 1

Review:  This is not the right product.  Predicated class: -1

(-1 = Negative, 0=Neutral, 1=Positive)


We used Amazon SageMaker Autopilot to automatically find the best model, hyper-parameters, and feature-engineering scripts for our dataset. Autopilot uses a uniquely-transparent approach to AutoML by generating re-usable Python scripts and notebooks.